In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import pytorch_lightning as pl
import torch
from ay2.torch.lightning.callbacks import (
    ACC_Callback,
    APCallback,
    AUC_Callback,
    Color_progress_bar,
    EER_Callback,
)
from config import get_cfg_defaults
from data.make_dataset import make_data
from models import LCNN_lit, RawNet2_lit

torch.set_float32_matmul_precision("medium")

In [3]:
cfg = get_cfg_defaults("config/experiments/RawNet2/wavefake_inner.yaml")
ds, dl = make_data(cfg.DATASET)

load model yaml in  config/experiments/RawNet2/0-model.yaml
        path  trainSet  method    fps  length     id
label                                               
0      91700     91700   91700  91700   91700  91700
1      13100     13100   13100  13100   13100  13100


In [4]:
model = RawNet2_lit()

In [4]:
model = LCNN_lit()

In [20]:
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu",
    devices=[1],
    check_val_every_n_epoch=1,
    logger=pl.loggers.CSVLogger(
        "/home/ay/data/Loggers/0-Audio", name="test", version=None
    ),
    default_root_dir="/home/ay/data/Loggers/0-Audio",
    callbacks=[
        Color_progress_bar(),
        ACC_Callback(batch_key="label", output_key="batch_pred"),
        AUC_Callback(batch_key="label", output_key="batch_pred"),
        EER_Callback(batch_key="label", output_key="batch_out")
    ]
    # limit_train_batches=0.1
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, train_dataloaders=dl.train, val_dataloaders=dl.val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type              ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model     │ RawNet2           │ 17.7 M │
│ 1 │ loss_fn   │ BCEWithLogitsLoss │      0 │
│ 2 │ acc_train │ BinaryAccuracy    │      0 │
│ 3 │ acc_val   │ BinaryAccuracy    │      0 │
│ 4 │ acc_test  │ BinaryAccuracy    │      0 │
│ 5 │ eer_train │ EER               │      0 │
│ 6 │ eer_val   │ EER               │      0 │
│ 7 │ eer_test  │ EER               │      0 │
└───┴───────────┴───────────────────┴────────┘

Trainable params: 17.7 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 17.7 M                                                                                               
Total estimated model params size (MB): 70

Output()

In [37]:
trainer.validate(model, dl.val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

start validation

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           loss            │    0.06355491280555725    │
│          val-acc          │    0.9742695689201355     │
└───────────────────────────┴───────────────────────────┘

[{'loss': 0.06355491280555725, 'val-acc': 0.9742695689201355}]

In [38]:
trainer.validate(model, dl.val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

start validation

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           loss            │    0.06355491280555725    │
│          val-acc          │    0.9743614792823792     │
└───────────────────────────┴───────────────────────────┘

[{'loss': 0.06355491280555725, 'val-acc': 0.9743614792823792}]

In [29]:
trainer.test(model, dl.test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           loss            │    0.16396945714950562    │
│         test-acc          │    0.9322580695152283     │
│         test_acc          │    0.9330637454986572     │
│         test_eer          │    0.07690182328224182    │
└───────────────────────────┴───────────────────────────┘

[{'loss': 0.16396945714950562,
  'test-acc': 0.9322580695152283,
  'test_acc': 0.9330637454986572,
  'test_eer': 0.07690182328224182}]